In [1]:
# 基本のライブラリを読み込む
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 統計モデルを推定するライブラリ
import statsmodels.api as sm
import statsmodels.tsa.api as tsa

# グラフを描画するライブラリ
from matplotlib import pylab as plt
import seaborn as sns
import matplotlib.dates as mdates

sns.set()

# グラフの日本語表記
from matplotlib import rcParams

rcParams["font.family"] = "IPAexGothic"
rcParams["font.sans-serif"] = "Meiryo"

In [2]:
# 表示設定
np.set_printoptions(linewidth=60)
pd.set_option("display.width", 80)

from matplotlib.pylab import rcParams

rcParams["figure.figsize"] = 8, 4

In [3]:
# データの読み込み
train_sample = pd.read_csv(
    "7-1-1-train-sample-1.csv", index_col="date", parse_dates=True, dtype="float"
)

# 日データであることの指定
train_sample.index.freq = "D"

# 結果の確認
print(train_sample.head(2))
print(train_sample.tail(2))

                     y
date                  
2000-05-01  182.079325
2000-05-02  141.334794
                     y
date                  
2000-08-07  264.246774
2000-08-08  214.887515


In [4]:
# ==============================================================
# 季節変動ありのローカル線形トレンドモデル（Local Linear Trend Model with Seasonality）
# ==============================================================

# UnobservedComponents（観測されない成分モデル）は、
# 状態空間モデル（State Space Model）をベースに、
# トレンド成分・季節成分・外生変数・誤差構造などを統合的に扱うクラスである。
#
# ここでは「ローカル線形トレンド」＋「季節変動（周期7）」をモデル化している。
# --------------------------------------------------------------

mod_sample = tsa.UnobservedComponents(
    endog=train_sample["y"],  # 対象の時系列データ
    level="local linear trend",  # 時間とともに変化する水準＋トレンド
    seasonal=7,  # 季節周期（7日周期の週次変動）
).fit()  # パラメータを最尤法（MLE）で推定

# ==============================================================
# 推定されたパラメータの確認
# ==============================================================

# .params で推定された分散パラメータなどを確認できる。
# 一般に、ローカル線形トレンドモデルでは以下が推定される：
#   - σ²_ε : 観測誤差（Irregular component）の分散
#   - σ²_η : 水準（Level component）の変動分散
#   - σ²_ζ : トレンド（Slope component）の変動分散
#   - （必要に応じて）季節成分の分散
#
# モデル構造の背後にある数式は次のように表される：
#
# 【状態空間表現】
# 観測方程式（Observation equation）:
#     y_t = μ_t + γ_t + ε_t,     ε_t ~ N(0, σ²_ε)
#
# 状態方程式（State equations）:
#     μ_t = μ_{t-1} + β_{t-1} + η_t,   η_t ~ N(0, σ²_η)
#     β_t = β_{t-1} + ζ_t,            ζ_t ~ N(0, σ²_ζ)
#     γ_t = -Σ_{i=1}^{s-1} γ_{t-i} + ω_t, ω_t ~ N(0, σ²_γ)   （季節性成分）
#
# ここで：
#   μ_t : 水準（Level）
#   β_t : トレンド（Slope）
#   γ_t : 季節性（Seasonal）
#   ε_t : 観測誤差
# --------------------------------------------------------------

print(mod_sample.params)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.34797D+00    |proj g|=  1.06009D-02

At iterate    5    f=  3.06172D+00    |proj g|=  7.34555D-02

At iterate   10    f=  2.96718D+00    |proj g|=  8.73561D-03

At iterate   15    f=  2.93725D+00    |proj g|=  3.43616D-02

At iterate   20    f=  2.90440D+00    |proj g|=  1.72459D-02

At iterate   25    f=  2.90263D+00    |proj g|=  5.42255D-04

At iterate   30    f=  2.90262D+00    |proj g|=  2.36588D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

 This problem is unconstrained.


In [5]:
# 予測
forecast_sample = mod_sample.forecast(7)

# 予測結果の確認
forecast_sample

2000-08-09    246.808962
2000-08-10    277.554573
2000-08-11    296.189758
2000-08-12    325.585518
2000-08-13    317.301612
2000-08-14    269.799431
2000-08-15    221.201080
Freq: D, Name: predicted_mean, dtype: float64